In [1]:
%%time
import pandas as pd
import datetime
import numpy as np 
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.linear_model import LinearRegression

def make_date(df):
  return datetime.date(df.YEAR, df.MONTH, df.DAY)

def make_features(data, max_lag, rolling_mean_size):
    data['year'] = data.dDate.dt.year
    data['month'] = data.dDate.dt.month
    data['day'] = data.dDate.dt.day
    data['dayofweek'] = data.dDate.dt.dayofweek
    for lag in range(1, max_lag + 1):
        data['lag_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)

airports = pd.read_csv('data/airports.csv')
airlines = pd.read_csv('data/airlines.csv')
flights = pd.read_csv('data/flights.csv', dtype={'ORIGIN_AIRPORT': str,'DESTINATION_AIRPORT': str } )

flights = flights.merge(airports[['IATA_CODE']],how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)
flights = flights.merge(airports[['IATA_CODE']],how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop('IATA_CODE',axis=1)

flights['dDate'] = flights.apply(make_date,axis = 1)

CPU times: user 2min 40s, sys: 36.2 s, total: 3min 16s
Wall time: 3min 30s


In [2]:
%%time
flight_grouped = flights.groupby(['dDate','DESTINATION_AIRPORT'])['ARRIVAL_DELAY'].sum().reset_index()
flight_grouped['dDate'] = pd.to_datetime(flight_grouped['dDate'])

CPU times: user 787 ms, sys: 183 ms, total: 969 ms
Wall time: 1.02 s


In [3]:
%%time
where_to_go = []
for dest in flight_grouped.DESTINATION_AIRPORT.unique():
    tempo = flight_grouped[flight_grouped.DESTINATION_AIRPORT==dest][['dDate','ARRIVAL_DELAY']]
    tempo.columns = ['dDate','ARRIVAL_DELAY']
    tempo = pd.DataFrame(tempo)
    
    try:
    
        make_features(tempo,21,7)
        tempo.dropna(inplace=True)
        tempo.set_index('dDate',inplace=True)
           
        X_train,X_test,y_train, y_test = train_test_split(tempo.drop('ARRIVAL_DELAY', axis=1),tempo.ARRIVAL_DELAY, shuffle=False, test_size=0.25)

        model_lr = LinearRegression()
        model_lr.fit(X_train,y_train)

        y_predicted_lr = model_lr.predict(X_test)
        where_to_go.append([dest,y_test.mean(),np.sqrt(mean_squared_error(y_test, y_predicted_lr))])

    except Exception as e:
        print('Error', str(e))
        

CPU times: user 7.66 s, sys: 79.1 ms, total: 7.74 s
Wall time: 7.81 s


In [4]:
%%time
where_to_go = pd.DataFrame(where_to_go)
where_to_go.columns = ['DESTINATION_AIRPORT','MEAN_ARRIVAL_DELAY_IN_PAST','RMSE']

CPU times: user 1.05 ms, sys: 193 µs, total: 1.24 ms
Wall time: 1.21 ms


In [5]:
%%time
start_airport = np.random.choice(flights['ORIGIN_AIRPORT'].unique()) 
where_to_go_from_start_airport = flights[flights.ORIGIN_AIRPORT==start_airport]['DESTINATION_AIRPORT'].unique()

where_to_go_from_start_airport = pd.DataFrame(where_to_go_from_start_airport)
where_to_go_from_start_airport.columns = ['DESTINATION_AIRPORT']

top3 = where_to_go_from_start_airport\
    .merge(where_to_go,on='DESTINATION_AIRPORT',how='inner')\
    .sort_values(by=['RMSE','MEAN_ARRIVAL_DELAY_IN_PAST'],ascending=[True, True])\
    .head(3)

print('Лучшие направления с аэропорта',start_airport)
print('')
print(top3)


Лучшие направления с аэропорта MEM

   DESTINATION_AIRPORT  MEAN_ARRIVAL_DELAY_IN_PAST         RMSE
21                 CVG                   80.379747   475.199263
14                 IAD                 -123.582278   726.147720
20                 TPA                  355.772152  1465.367442
CPU times: user 1.33 s, sys: 427 ms, total: 1.76 s
Wall time: 1.89 s
